# Compute PSD and Z-Score of Pilot Data

## Goals:
1. **Data Import**
    - Import the preprocessed data from npz and json files

2. **Feature Extraction**
    - Average the time series data for each epoch over the occipital channels
    - Calculate the Power Spectral Density (PSD) for each epoch.
    - Calculate the Z-score for each epoch (used to determine signal cutoff):
        * **Formula**:   PSD Z-score = (PSD(single trial) - Mean PSD of baseline trials) / Std PSD of baseline trials
    - Find a way to combine the Z-scores for each stimulus (will end up with n_zscores = n_epochs per stimulus)
        
3. **Data Formatting**
    - Save the Z score data and export to an excel sheet

In [1]:
def get_band_average(pxx, f_values, band):
    """Calculate the average power spectral density in a specified frequency band.
    
    Args:
        pxx: Power spectral density values (1D or 2D array)
        f_values: Frequency values corresponding to pxx (1D array)
        band: List of [low_freq, high_freq] defining the band
        
    Returns:
        Mean PSD in the specified band
    """
    band_mask = (f_values >= band[0]) & (f_values <= band[1])
    
    # For 1D array (single epoch)
    if len(pxx.shape) == 1:
        return np.mean(pxx[band_mask])
    # For 2D array (multiple epochs)
    else:
        return np.mean(pxx[:, band_mask], axis=1)

# Import Libraries

In [2]:
# Standard libraries
import json
import numpy as np
import pandas as pd
import scipy.signal as signal
import matplotlib.pyplot as plt

# Custom libraries
from Functions import processing

%matplotlib qt

# Import Epoched Data and Settings

In [3]:
# Load list of files to import
files = [  
    "sub-P009_ses-S001_task-T1_run-001_eeg",
    #"sub-P002_ses-S001_task-T1_run-001_eeg",
    #"sub-P003_ses-S001_task-T1_run-001_eeg",
    #"sub-P004_ses-S001_task-T1_run-001_eeg",
    #"sub-P005_ses-S001_task-T1_run-001_eeg",
    #"sub-P006_ses-S001_task-T1_run-001_eeg",
    #"sub-P008-Redo_ses-S001_task-T2_run-001_eeg", 
    #"sub-P009_ses-S001_task-T1_run-001_eeg",
    #"sub-P010_ses-S001_task-T1_run-001_eeg", 
]

# Get unique subject IDs- Currently not used
subject_ids = [file.split('_')[0] for file in files]
unique_subject_ids = list(set(subject_ids))

# Preallocate variables to store EEG data and settings
eeg_epochs = [None] * len(files)
settings = [None] * len(files)
baseline = [None] * len(files)

# Import data
for f, file in enumerate(files):
    # Import EEG data, since it is stored in a compressed numpy file (.npz) we need to use the np.load function 
    loaded_data = np.load(f"Data\\Pilot-Data\\EEG\\All\\{file}.npz", allow_pickle=True)

    # Access the data for each stimulus
    eeg_epochs[f] = {stim_label: loaded_data[stim_label] for stim_label in loaded_data.files}

    # Import settings
    with open(f"Data\\Pilot-Data\\EEG\\All\\{file}.json", "r") as file_object:
        settings[f] = json.load(file_object)

    # Import baseline data
    loaded_baseline = np.load(f"Data\\Pilot-Data\\EEG\\All\\{file}_baseline.npz", allow_pickle=True)
    baseline[f] = {stim_label: loaded_baseline[stim_label] for stim_label in loaded_baseline.files}

# Average time series across occipital channels per epoch for "on" data and baseline data

In [4]:
selected_channels = ['O1', 'Oz', 'O2']
occipital_epochs = [{} for _ in range(len(files))]  # List of dictionaries for each file
occipital_baseline = [{} for _ in range(len(files))]  # List of dictionaries for each file

for f in range(len(files)):
    # Get channel names for this file
    available_channels = settings[f]['new_ch_names']
    
    # Get indices of channels that are actually available
    channel_indices = []
    for ch in selected_channels:
        if ch in available_channels:
            channel_indices.append(available_channels.index(ch))
        else:
            print(f"Channel {ch} is not available in file {files[f]}")

    if not channel_indices:
        print(f"No selected occipital channels found for file {files[f]}. Skipping averaging.")
        continue  # Skip this file if no channels are available

    # Process epochs
    for stim_label in eeg_epochs[f]:
        # Extract available occipital channels
        occipital_epochs[f][stim_label] = eeg_epochs[f][stim_label][:, channel_indices, :]
        # Average over those channels
        occipital_epochs[f][stim_label] = np.mean(occipital_epochs[f][stim_label], axis=1)

    # Process baseline
    for stim_label in baseline[f]:
        stim_epochs = baseline[f][stim_label]

        occipital_baseline[f][stim_label] = baseline[f][stim_label][:, channel_indices, :]
        occipital_baseline[f][stim_label] = np.mean(occipital_baseline[f][stim_label], axis=1)

# Compute PSD for each "On" Epoch

In [5]:
# PSD settings
window_size = 10  # 10 = 0.1 Hz resolution, 5 = 0.2 Hz resolution, 2 = 0.5 Hz resolution

# Preallocate variables
eeg_f = [None] * len(files)
eeg_pxx = [None] * len(files)  # Preallocate to list in case not all files have the same number of epochs

# Compute PSD for each file
for f in range(len(files)):
    eeg_f[f] = {}
    eeg_pxx[f] = {}

    # Compute PSD for each stimulus
    for stim_label, epochs in occipital_epochs[f].items(): 
        eeg_f[f][stim_label] = []
        eeg_pxx[f][stim_label] = []

        # Compute PSD for each epoch
        for epoch in epochs:  # Each epoch is now a 1D array (num_samples,)
            f_values, pxx_values = signal.welch(
                x=epoch,  # 1D array (samples,)
                fs=settings[f]["eeg_srate"],
                nperseg=window_size * settings[f]["eeg_srate"],
                noverlap=(window_size * settings[f]["eeg_srate"]) * 0.5,  # 50% overlap
            )
            eeg_f[f][stim_label].append(f_values)
            eeg_pxx[f][stim_label].append(pxx_values)

        # Convert lists to arrays for consistency
        eeg_f[f][stim_label] = np.array(eeg_f[f][stim_label])  # Shape: (num_epochs, num_frequencies)
        eeg_pxx[f][stim_label] = np.array(eeg_pxx[f][stim_label])  # Shape: (num_epochs, num_frequencies)

#Print the shape of the PSD results for each stimulus
for f, file in enumerate(files):
     print(f"File: {file}")
     for stim_label in eeg_f[f].keys():
         print(f"  Stimulus: {stim_label}, PSD shape: {eeg_pxx[f][stim_label].shape}")

File: sub-P009_ses-S001_task-T1_run-001_eeg
  Stimulus: Contrast1Size1, PSD shape: (4, 1281)
  Stimulus: Contrast1Size2, PSD shape: (3, 1281)
  Stimulus: Contrast1Size3, PSD shape: (2, 1281)
  Stimulus: Contrast2Size1, PSD shape: (2, 1281)
  Stimulus: Contrast2Size2, PSD shape: (3, 1281)
  Stimulus: Contrast2Size3, PSD shape: (2, 1281)
  Stimulus: Contrast3Size1, PSD shape: (5, 1281)
  Stimulus: Contrast3Size2, PSD shape: (3, 1281)
  Stimulus: Contrast3Size3, PSD shape: (3, 1281)
  Stimulus: Contrast4Size1, PSD shape: (5, 1281)
  Stimulus: Contrast4Size2, PSD shape: (3, 1281)
  Stimulus: Contrast4Size3, PSD shape: (5, 1281)


# Plot PSD of "On" Epochs

In [6]:
plot_psd = True  # Enable to see plots
f_limits = [9.5, 10.5]  # Frequency limits for the plots [min, max][Hz]
file_to_plot = 0    # Select index of file to be plotted
num_stimuli = 12    # Number of stimuli

if plot_psd:
    fig, axes = plt.subplots(3, 4, figsize=(15, 10))
    fig.suptitle(f'PSD for All Stimuli in File: {files[file_to_plot]}')
    axes = axes.flatten()

    for stim_idx in range(num_stimuli):
        stim_label = list(eeg_f[file_to_plot].keys())[stim_idx]
        
        # Extract frequency values
        fmask = (eeg_f[file_to_plot][stim_label][0] >= f_limits[0]) & (eeg_f[file_to_plot][stim_label][0] <= f_limits[1])
        temp_freq = eeg_f[file_to_plot][stim_label][0][fmask]
        
        # Plot each epoch separately for this stimulus
        for epoch_idx, epoch in enumerate(eeg_pxx[file_to_plot][stim_label]):
            avg_pxx = epoch[fmask]  # Get values within frequency limits
            axes[stim_idx].plot(temp_freq, avg_pxx, label=f'Epoch {epoch_idx+1}')  # Label each epoch
        
        # Set plot details
        axes[stim_idx].set_title(f'Stimulus: {stim_label}')
        axes[stim_idx].set_xlim(f_limits)
        axes[stim_idx].set_ylabel("PXX [$\mu$V$^2$/Hz]")

        if stim_idx >= 8:
            axes[stim_idx].set_xlabel("Frequency [Hz]")

        # Add legend to identify epochs
        axes[stim_idx].legend()

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

<>:26: SyntaxWarning: invalid escape sequence '\m'
<>:26: SyntaxWarning: invalid escape sequence '\m'
C:\Users\admin\AppData\Local\Temp\ipykernel_5276\3634214831.py:26: SyntaxWarning: invalid escape sequence '\m'
  axes[stim_idx].set_ylabel("PXX [$\mu$V$^2$/Hz]")


# Average PSD over epochs for each stimulus and plot

In [7]:
# average PSD across epochs for each stimulus
eeg_pxx_mean = [{} for _ in range(len(files))]  # Preallocate to list in case not all files have the same number of epochs

for f in range(len(files)):
    for stim_label in eeg_f[f]:
        # Average the PSD across epochs
        avg_pxx = np.mean(eeg_pxx[f][stim_label], axis=0)
        eeg_pxx_mean[f][stim_label] = avg_pxx  # Replace with averaged PSD

# Plot averaged PSD for each stimulus
if plot_psd:
    fig, axes = plt.subplots(3, 4, figsize=(15, 10))
    fig.suptitle(f'Average PSD for All Stimuli in File: {files[file_to_plot]}')
    axes = axes.flatten()

    for stim_idx in range(num_stimuli):
        stim_label = list(eeg_f[file_to_plot].keys())[stim_idx]
        
        # Extract frequency values
        fmask = (eeg_f[file_to_plot][stim_label][0] >= f_limits[0]) & (eeg_f[file_to_plot][stim_label][0] <= f_limits[1])
        temp_freq = eeg_f[file_to_plot][stim_label][0][fmask]
        
        # Plot the averaged PSD for this stimulus
        avg_pxx = eeg_pxx_mean[file_to_plot][stim_label][fmask]  # Get values within frequency limits
        axes[stim_idx].plot(temp_freq, avg_pxx, label='Average')  # Label as average
        
        # Set plot details
        axes[stim_idx].set_title(f'Stimulus: {stim_label}')
        axes[stim_idx].set_xlim(f_limits)
        axes[stim_idx].set_ylabel("PXX [$\mu$V$^2$/Hz]")

        if stim_idx >= 8:
            axes[stim_idx].set_xlabel("Frequency [Hz]")

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()


<>:30: SyntaxWarning: invalid escape sequence '\m'
<>:30: SyntaxWarning: invalid escape sequence '\m'
C:\Users\admin\AppData\Local\Temp\ipykernel_5276\3882238461.py:30: SyntaxWarning: invalid escape sequence '\m'
  axes[stim_idx].set_ylabel("PXX [$\mu$V$^2$/Hz]")


# Compute Mean and STD of the PSD for Baseline Data

In [8]:
# PSD settings
window_size = 10  # 10 = 0.1 Hz resolution, 5 = 0.2 Hz resolution, 2 = 0.5 Hz resolution

# Preallocate variables
baseline_f = [None] * len(files)
baseline_pxx = [None] * len(files)  # Preallocate to list in case not all files have the same number of epochs

# Compute PSD for each file
for f in range(len(files)):
    baseline_f[f] = {}
    baseline_pxx[f] = {}

    # Compute PSD for each stimulus
    for stim_label, epochs in occipital_baseline[f].items(): 
        baseline_f[f][stim_label] = []
        baseline_pxx[f][stim_label] = []

        # Compute PSD for each epoch
        for epoch in epochs:  # Each epoch is now a 1D array (num_samples,)
            f_values, pxx_values = signal.welch(
                x=epoch,  # 1D array (samples,)
                fs=settings[f]["eeg_srate"],
                nperseg=window_size * settings[f]["eeg_srate"],
                noverlap=(window_size * settings[f]["eeg_srate"]) * 0.5,  # 50% overlap
            )
            baseline_f[f][stim_label].append(f_values)
            baseline_pxx[f][stim_label].append(pxx_values)

        # Convert lists to arrays for consistency
        baseline_f[f][stim_label] = np.array(baseline_f[f][stim_label])  # Shape: (num_epochs, num_frequencies)
        baseline_pxx[f][stim_label] = np.array(baseline_pxx[f][stim_label])  # Shape: (num_epochs, num_frequencies)

        # get the mean and std of the baseline data
        baseline_mean = np.mean(baseline_pxx[f][stim_label], axis=0)
        baseline_std = np.std(baseline_pxx[f][stim_label], axis=0)

# Get average PSD for 2 frequency bands
- mean PSD per stimulus between 9.5-10.5 Hz and 19.5-20.5 Hz

In [9]:
# Define frequency bands
band_1 = [9.8, 10.2]  # 9-11 Hz band
band_2 = [19.8, 20.2]  # 19-21 Hz band

# Initialize storage
band_averages = [{} for _ in range(len(files))]

for f in range(len(files)):
    for stim_label in eeg_f[f].keys():
        # Get PSD and frequency values for this stimulus
        current_psd = eeg_pxx_mean[f][stim_label]
        current_freqs = eeg_f[f][stim_label][0]  # Assuming this is 1D array
        
        # Calculate band averages
        band_averages[f][stim_label] = {
            'band_1': get_band_average(current_psd, current_freqs, band_1),
            'band_2': get_band_average(current_psd, current_freqs, band_2)
        }

# Print results
for f, file in enumerate(files):
    print(f"\nFile: {file}")
    for stim_label, bands in band_averages[f].items():
        # Handle both scalar and array outputs
        b1 = np.mean(bands['band_1'])  # In case it's an array
        b2 = np.mean(bands['band_2'])
        print(f"  Stimulus: {stim_label}")
        print(f"    Band 1 (9.5-10.5 Hz) avg: {b1:.4f}")
        print(f"    Band 2 (19.5-20.5 Hz) avg: {b2:.4f}")


File: sub-P009_ses-S001_task-T1_run-001_eeg
  Stimulus: Contrast1Size1
    Band 1 (9.5-10.5 Hz) avg: 11.3534
    Band 2 (19.5-20.5 Hz) avg: 2.1121
  Stimulus: Contrast1Size2
    Band 1 (9.5-10.5 Hz) avg: 27.3400
    Band 2 (19.5-20.5 Hz) avg: 3.0915
  Stimulus: Contrast1Size3
    Band 1 (9.5-10.5 Hz) avg: 38.4921
    Band 2 (19.5-20.5 Hz) avg: 11.3862
  Stimulus: Contrast2Size1
    Band 1 (9.5-10.5 Hz) avg: 12.8341
    Band 2 (19.5-20.5 Hz) avg: 1.2383
  Stimulus: Contrast2Size2
    Band 1 (9.5-10.5 Hz) avg: 28.4070
    Band 2 (19.5-20.5 Hz) avg: 5.8759
  Stimulus: Contrast2Size3
    Band 1 (9.5-10.5 Hz) avg: 16.9095
    Band 2 (19.5-20.5 Hz) avg: 3.3085
  Stimulus: Contrast3Size1
    Band 1 (9.5-10.5 Hz) avg: 7.7965
    Band 2 (19.5-20.5 Hz) avg: 2.1779
  Stimulus: Contrast3Size2
    Band 1 (9.5-10.5 Hz) avg: 7.7503
    Band 2 (19.5-20.5 Hz) avg: 2.7429
  Stimulus: Contrast3Size3
    Band 1 (9.5-10.5 Hz) avg: 11.4813
    Band 2 (19.5-20.5 Hz) avg: 4.4636
  Stimulus: Contrast4Size1
  

# Baseline mean and STD averages for 2 frequency bands

In [ ]:
#get band averages for baseline data mean
baseline_band_averages = [{} for _ in range(len(files))]

for f in range(len(files)):
    for stim_label in baseline_f[f].keys():
        # Get PSD and frequency values for this stimulus using baseline_mean and baseline_f
        current_psd = baseline_mean  # Use the mean PSD for baseline
        current_freqs = baseline_f[f][stim_label][0]
        
        # Calculate band averages
        baseline_band_averages[f][stim_label] = {
            'band_1': get_band_average(current_psd, current_freqs, band_1),
            'band_2': get_band_average(current_psd, current_freqs, band_2)
        }

#do the same for the baseline_std
baseline_band_std = [{} for _ in range(len(files))]

for f in range(len(files)):
    for stim_label in baseline_f[f].keys():
        # Get PSD and frequency values for this stimulus using baseline_std and baseline_f
        current_psd = baseline_std  # Use the std PSD for baseline
        current_freqs = baseline_f[f][stim_label][0]
        
        # Calculate band averages
        baseline_band_std[f][stim_label] = {
            'band_1': get_band_average(current_psd, current_freqs, band_1),
            'band_2': get_band_average(current_psd, current_freqs, band_2)
        }

# Print baseline band averages and stds
for f, file in enumerate(files):
    print(f"\nBaseline File: {file}")
    for stim_label, bands in baseline_band_averages[f].items():
        # Handle both scalar and array outputs
        b1 = np.mean(bands['band_1'])  # In case it's an array
        b2 = np.mean(bands['band_2'])
        print(f"  Stimulus: {stim_label}")
        print(f"    Baseline Band 1 (9.5-10.5 Hz) avg: {b1:.4f}")
        print(f"    Baseline Band 2 (19.5-20.5 Hz) avg: {b2:.4f}")

# Print baseline band stds
for f, file in enumerate(files):
    print(f"\nBaseline Std File: {file}")
    for stim_label, bands in baseline_band_std[f].items():
        # Handle both scalar and array outputs
        b1 = np.mean(bands['band_1'])  # In case it's an array
        b2 = np.mean(bands['band_2'])
        print(f"  Stimulus: {stim_label}")
        print(f"    Baseline Band 1 (9.5-10.5 Hz) std: {b1:.4f}")
        print(f"    Baseline Band 2 (19.5-20.5 Hz) std: {b2:.4f}\n")

check = False  # Enable to see if the band averages are correct

if check:
    mean_check_1 = np.mean(baseline_mean[95:106])
    print(f"Baseline Mean Check Band 1: {mean_check_1:.4f}")
    mean_check_2 = np.mean(baseline_mean[195:206])
    print(f"Baseline Mean Check Band 2: {mean_check_2:.4f}\n")
    std_check_1 = np.mean(baseline_std[95:106])
    print(f"Baseline Standard Deviation Check Band 1: {std_check_1:.4f}")
    std_check_2 = np.mean(baseline_std[195:206])
    print(f"Baseline Standard Deviation Check Band 2: {std_check_2:.4f}")


Baseline File: sub-P009_ses-S001_task-T1_run-001_eeg
  Stimulus: stimulus ended
    Baseline Band 1 (9.5-10.5 Hz) avg: 4.7083
    Baseline Band 2 (19.5-20.5 Hz) avg: 0.6358

Baseline Std File: sub-P009_ses-S001_task-T1_run-001_eeg
  Stimulus: stimulus ended
    Baseline Band 1 (9.5-10.5 Hz) std: 5.0402
    Baseline Band 2 (19.5-20.5 Hz) std: 0.6009

Baseline Mean Check Band 1: 4.9976
Baseline Mean Check Band 2: 0.6453

Baseline Standard Deviation Check Band 1: 6.4159
Baseline Standard Deviation Check Band 2: 0.6831


# Calculate Z Score for all epochs

In [ ]:
#calaulye the z-score for each band and stimulus
z_scores = [{} for _ in range(len(files))]
    
# Get the baseline band average for each file
for stim_label, bands in baseline_band_averages[f].items():
    baseline_band_1_avg = np.mean(bands['band_1'])  
    baseline_band_2_avg = np.mean(bands['band_2'])

# Get the baseline band std for each file
for stim_label, bands in baseline_band_std[f].items():
    baseline_band_1_std = np.mean(bands['band_1'])  
    baseline_band_2_std = np.mean(bands['band_2'])

# Calculate z-scores for each stimulus and band
for f in range(len(files)):
    for stim_label in band_averages[f].keys():
        # Get band averages for the current stimulus
        band_1_avg = band_averages[f][stim_label]['band_1']
        band_2_avg = band_averages[f][stim_label]['band_2']
        
        # Calculate z-scores
        z_scores[f][stim_label] = {
            'band_1_z': (band_1_avg - baseline_band_1_avg) / (baseline_band_1_std),
            'band_2_z': (band_2_avg - baseline_band_2_avg) / (baseline_band_2_std)
        }
    
# Print z-scores
for f, file in enumerate(files):
    print(f"\nZ-Scores for File: {file}")
    for stim_label, scores in z_scores[f].items():
        print(f"  Stimulus: {stim_label}")
        print(f"    Band 1 (9.5-10.5 Hz) z-score: {scores['band_1_z']:.4f}")
        print(f"    Band 2 (19.5-20.5 Hz) z-score: {scores['band_2_z']:.4f}")

check = False

if check:
    # Check if the score are correct for the first stimulus
    first_stimulus = list(z_scores[0].keys())[0]
    print(f"\nZ-Score Check for First Stimulus: {first_stimulus}")
    print(f"  Band 1 (9.5-10.5 Hz) z-score: {z_scores[0][first_stimulus]['band_1_z']:.4f}")
    print(f"  Band 2 (19.5-20.5 Hz) z-score: {z_scores[0][first_stimulus]['band_2_z']:.4f}")

    check_band_1 = (band_averages[0][first_stimulus]['band_1'] - baseline_band_1_avg) / baseline_band_1_std
    print(f"  Check Band 1 Z-Score: {check_band_1:.4f}")
    check_band_2 = (band_averages[0][first_stimulus]['band_2'] - baseline_band_2_avg) / baseline_band_2_std
    print(f"  Check Band 2 Z-Score: {check_band_2:.4f}")


Z-Scores for File: sub-P009_ses-S001_task-T1_run-001_eeg
  Stimulus: Contrast1Size1
    Band 1 (9.5-10.5 Hz) z-score: 1.3184
    Band 2 (19.5-20.5 Hz) z-score: 2.4568
  Stimulus: Contrast1Size2
    Band 1 (9.5-10.5 Hz) z-score: 4.4902
    Band 2 (19.5-20.5 Hz) z-score: 4.0868
  Stimulus: Contrast1Size3
    Band 1 (9.5-10.5 Hz) z-score: 6.7028
    Band 2 (19.5-20.5 Hz) z-score: 17.8913
  Stimulus: Contrast2Size1
    Band 1 (9.5-10.5 Hz) z-score: 1.6122
    Band 2 (19.5-20.5 Hz) z-score: 1.0027
  Stimulus: Contrast2Size2
    Band 1 (9.5-10.5 Hz) z-score: 4.7019
    Band 2 (19.5-20.5 Hz) z-score: 8.7208
  Stimulus: Contrast2Size3
    Band 1 (9.5-10.5 Hz) z-score: 2.4208
    Band 2 (19.5-20.5 Hz) z-score: 4.4479
  Stimulus: Contrast3Size1
    Band 1 (9.5-10.5 Hz) z-score: 0.6127
    Band 2 (19.5-20.5 Hz) z-score: 2.5663
  Stimulus: Contrast3Size2
    Band 1 (9.5-10.5 Hz) z-score: 0.6035
    Band 2 (19.5-20.5 Hz) z-score: 3.5067
  Stimulus: Contrast3Size3
    Band 1 (9.5-10.5 Hz) z-score: 

# Plot Z-scores

In [ ]:
plot_zscore = True  # Enable to see z-score plots

if plot_zscore:
    fig, axes = plt.subplots(3, 4, figsize=(15, 10))
    fig.suptitle(f'Z-Scores for All Stimuli in File: {files[file_to_plot]}')
    axes = axes.flatten()

    for stim_idx in range(num_stimuli):
        stim_label = list(z_scores[file_to_plot].keys())[stim_idx]
        
        # Extract z-scores
        band_1_z = z_scores[file_to_plot][stim_label]['band_1_z']
        band_2_z = z_scores[file_to_plot][stim_label]['band_2_z']
        
        # Plot z-scores as bar plots
        axes[stim_idx].bar(['Band 1 (9.5-10.5 Hz)', 'Band 2 (19.5-20.5 Hz)'], [band_1_z, band_2_z])
        
        # Set plot details
        axes[stim_idx].set_title(f'Stimulus: {stim_label}')
        axes[stim_idx].set_ylabel("Z-Score")
        
        if stim_idx >= 8:
            axes[stim_idx].set_xlabel("Frequency Band")

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()


# Export Z-scores

In [ ]:
save_zscore = True  # Set to True to save the z-scores to a CSV file

if save_zscore:
    # Get all unique stimuli across files (assuming same stimuli in each file)
    stimuli = list(z_scores[0].keys())  # Gets stimuli from first file
    
    # Prepare data for DataFrame
    data = {
        'Stimulus': [],
        '9-11Hz_zscore': [],
        '19-21Hz_zscore': []
    }
    
    # For each stimulus, collect z-scores from all files
    for stim in stimuli:
        # Initialize lists for this stimulus
        band1_scores = []
        band2_scores = []
        
        # Collect scores from each file
        for f in range(len(z_scores)):
            if stim in z_scores[f]:  # Check if stimulus exists in this file
                band1_scores.append(z_scores[f][stim]['band_1_z'])
                band2_scores.append(z_scores[f][stim]['band_2_z'])
        
        # Take mean across files for this stimulus
        data['Stimulus'].append(stim)
        data['9-11Hz_zscore'].append(np.mean(band1_scores))
        data['19-21Hz_zscore'].append(np.mean(band2_scores))
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Save to CSV (modify path as needed)
    save_path = 'Data\\Pilot-Data\\EEG\\zscores_P009_bands.csv'
    df.to_csv(save_path, index=False)

Z-scores saved to Data\Pilot-Data\EEG\zscores_P009_bands.csv
